In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import copy
import shap
import pickle
import os

from scipy import linalg
from scipy.special import expit
from scipy import stats
from tqdm import tqdm
from matplotlib import cm
from sklearn.base import TransformerMixin, ClassifierMixin
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, 
                              RandomForestRegressor, GradientBoostingRegressor)
from sklearn.model_selection import KFold
from sklearn.metrics import (roc_auc_score, f1_score, precision_score, recall_score, 
                             RocCurveDisplay, PrecisionRecallDisplay, 
                             mean_squared_error)
from sklearn.preprocessing import OneHotEncoder
from pandas.api.types import CategoricalDtype

In [2]:
# Import modelling functions
from ensemble_functions import *
from weighting_functions import *
from explainer_functions import *

## Load dataset

### Imputed using MICE

In [3]:
# Data parameters
imputed_file = "imputed_r.pickle"
mimic_dir = "../../data/mimic-iii/"
Ns = [20000, 10000, 5000, 2000, 1000]

# Place to store results
results_path = "../../results/metrics/"
if not os.path.exists(results_path):
    os.mkdir(results_path)
figures_path = "../../results/figures/gbt_models_rpy2/"
if not os.path.exists(figures_path):
    os.mkdir(figures_path)
model_iter_name = "_rpy2"

# Load imputed dataset
with open(mimic_dir + imputed_file, "rb") as handle:
    mimic_imputed = pickle.load(handle)

In [4]:
# Process the categorical variables
# Note: as we do not need the columns of missingflag to align, we only modify the data
# but do note that this may cause bug in future
ohe_ethnicity = OneHotEncoder(drop="first", sparse=False).fit(
    mimic_imputed[Ns[0]]["imp"][0][["ETHNICITY"]])
ohe_gender = OneHotEncoder(drop="first", sparse=False).fit(
    mimic_imputed[Ns[0]]["imp"][0][["GENDER"]]
)
ohe_marital = OneHotEncoder(drop="first", sparse=False).fit(
    mimic_imputed[Ns[0]]["imp"][0][["MARITALSTATUS"]])

for n in Ns:
    for m, imp in enumerate(mimic_imputed[n]["imp"]):
        # Transform gender column
        gender_temp = ohe_gender.transform(imp[["GENDER"]])
        gender_temp = pd.DataFrame(gender_temp, columns=ohe_gender.get_feature_names_out(), 
                                   index=imp.index)
        mimic_imputed[n]["imp"][m] = pd.concat(
            [mimic_imputed[n]["imp"][m], gender_temp], axis=1
        ).drop(["GENDER"], axis=1)
        
        # Transform ethnicity column
        eth_temp = ohe_ethnicity.transform(imp[["ETHNICITY"]])
        eth_temp = pd.DataFrame(eth_temp, columns=ohe_ethnicity.get_feature_names_out(), 
                                index=imp.index)
        mimic_imputed[n]["imp"][m] = pd.concat(
            [mimic_imputed[n]["imp"][m], eth_temp], axis=1
        ).drop(["ETHNICITY"], axis=1)
        
        # Transform marital status column
        marital_temp = ohe_marital.transform(imp[["MARITALSTATUS"]])
        marital_temp = pd.DataFrame(marital_temp, columns=ohe_marital.get_feature_names_out(), 
                                    index=imp.index)
        mimic_imputed[n]["imp"][m] = pd.concat(
            [mimic_imputed[n]["imp"][m], marital_temp], axis=1
        ).drop(["MARITALSTATUS"], axis=1)

### Complete case data

In [5]:
# Load complete case data
mimic_cc = {}
for n in Ns:
    mimic_cc[n] = pd.read_csv(mimic_dir + "complete_case_{}.csv".format(n))

In [6]:
# Process the categorical variables
for n in Ns:
    # Transform gender column
    gender_temp = ohe_gender.transform(mimic_cc[n][["GENDER"]])
    gender_temp = pd.DataFrame(gender_temp, columns=ohe_gender.get_feature_names_out(), 
                               index=mimic_cc[n].index)
    mimic_cc[n] = pd.concat(
        [mimic_cc[n], gender_temp], axis=1
    ).drop(["GENDER"], axis=1)

    # Transform ethnicity column
    eth_temp = ohe_ethnicity.transform(mimic_cc[n][["ETHNICITY"]])
    eth_temp = pd.DataFrame(eth_temp, columns=ohe_ethnicity.get_feature_names_out(), 
                            index=mimic_cc[n].index)
    mimic_cc[n] = pd.concat(
        [mimic_cc[n], eth_temp], axis=1
    ).drop(["ETHNICITY"], axis=1)

    # Transform marital status column
    marital_temp = ohe_marital.transform(mimic_cc[n][["MARITALSTATUS"]])
    marital_temp = pd.DataFrame(marital_temp, columns=ohe_marital.get_feature_names_out(), 
                                index=mimic_cc[n].index)
    mimic_cc[n] = pd.concat(
        [mimic_cc[n], marital_temp], axis=1
    ).drop(["MARITALSTATUS"], axis=1)

## Model development and assessment

### Data preparation

In [7]:
# Determine covariates and outcome variables
yvar = "READM90D"
Xvars = mimic_cc[Ns[0]].drop("READM90D", axis=1).columns.values

# Selected features for model explanation
selected_expl = []

In [8]:
# Get readmission % in each dataset
for n in Ns:
    print("Dataset {}, number of readmissions = {} / {}".format(
        n, mimic_imputed[n]["imp"][0][yvar].sum(), n
    ))

Dataset 20000, number of readmissions = 1595 / 20000
Dataset 10000, number of readmissions = 785 / 10000
Dataset 5000, number of readmissions = 391 / 5000
Dataset 2000, number of readmissions = 155 / 2000
Dataset 1000, number of readmissions = 75 / 1000


In [9]:
# Get readmission % in each dataset
for n in Ns:
    print("Dataset {} (CC), number of readmissions = {} / {}".format(
        n, mimic_cc[n][yvar].sum(), mimic_cc[n].shape[0]
    ))

Dataset 20000 (CC), number of readmissions = 1006 / 11740
Dataset 10000 (CC), number of readmissions = 495 / 5836
Dataset 5000 (CC), number of readmissions = 252 / 2917
Dataset 2000 (CC), number of readmissions = 98 / 1164
Dataset 1000 (CC), number of readmissions = 49 / 571


### General model setup (GBT)

In [10]:
# Cross-validation folds
n_splits = 5

# Set random seed
SEED = 2023

# Probability cutoff to compute metrics such as F1 score
pred_cutoff = np.linspace(0.1, 1, 10)

## For now, we are using the default setup from sklearn
basemdl = GradientBoostingClassifier(random_state=SEED)
basemdlname = "gbt"

In [11]:
# Placeholder tables to store performance metrics (AUROC and F1)
mimic_auroc = pd.DataFrame(np.zeros((len(Ns), 3)), index=Ns, 
                            columns=["CC", "Ensemble", "Weighting"])
mimic_f1 = {}
for c in ["CC", "Ensemble", "Weighting"]:
    mimic_f1[c] = pd.DataFrame(np.zeros((len(Ns), len(pred_cutoff))), 
                               index=Ns, columns=pred_cutoff)

### Complete case data

#### Cross-validation

In [12]:
# Iterate over all versions of the data
for n in tqdm(Ns):
    # Separate indep and outcome variables
    X = mimic_cc[n][Xvars]
    y = mimic_cc[n][yvar]
    
    # Initialise k-fold object
    kf = KFold(n_splits=n_splits, random_state=SEED, shuffle=True)
    
    # Placeholder for predictions to calculate performance metrics
    preds = []
    
    # Set random seed for np.random
    np.random.seed(SEED)
    
    # Iterate over the folds
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        # Get train and test data
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]
        
        # Due to low % of readmissions, need to do resampling of minority class
        train_index_pos = y_train[y_train == 1].index
        train_index_neg = y_train[y_train == 0].index
        train_index_pos_up = np.random.choice(
            train_index_pos, size=len(train_index_neg), replace=True
        )
        train_index_up = np.concatenate((train_index_pos_up, train_index_neg), axis=0)
        X_train_up = X_train.loc[train_index_up]
        y_train_up = y_train.loc[train_index_up]
        
        # Train classifier on training data
        rf = copy.deepcopy(basemdl).fit(X_train_up, y_train_up)
        
        # Predict on test data and store predictions
        pred_ = rf.predict_proba(X_test)[:, 1]
        preds.append(pd.DataFrame({"true": y_test, "pred": pred_}))
    
    # Aggregate predictions, compute AUROC and F1 score
    preds = pd.concat(preds)
    for p in pred_cutoff:
        preds["pred_labels"] = preds["pred"] > p
        mimic_f1["CC"].loc[n, p] = f1_score(preds["true"], preds["pred_labels"])
    mimic_auroc.loc[n, "CC"] = roc_auc_score(preds["true"], preds["pred"])

100%|█████████████████████████████████████████████| 5/5 [00:08<00:00,  1.68s/it]


#### Full model training and explanation

In [13]:
%matplotlib agg
# Iterate over all versions of the data
for n in Ns:
    # Separate indep and outcome variables
    X = mimic_cc[n][Xvars]
    y = mimic_cc[n][yvar]
    
    # Due to low % of readmissions, need to do resampling of minority class
    index_pos = y[y == 1].index
    index_neg = y[y == 0].index
    index_pos_up = np.random.choice(
        index_pos, size=len(index_neg), replace=True
    )
    index_up = np.concatenate((index_pos_up, index_neg), axis=0)
    X_up = X.loc[index_up]
    y_up = y.loc[index_up]
    
    # Train RF classifier
    rf = copy.deepcopy(basemdl).fit(X_up, y_up)
    
    # Create output path for model explanation
    expl_path = figures_path + "mimic_cc_{}/".format(n)
    if not os.path.exists(expl_path):
        os.mkdir(expl_path)
    
    # Model explanation
    background_data = shap.maskers.Independent(X, max_samples=100)
    pred_fn = lambda x: rf.predict_proba(x)[:, 1]
    expl = shap.Explainer(pred_fn, background_data, link=shap.links.logit)
    
    # Calculate SHAP values
    shapvals = expl(X)
    
    # Create SHAP dependence plots for selected features
    for c in selected_expl:
        f1, ax1 = plt.subplots(figsize=(5, 5), ncols=1, nrows=1)
        # shap.dependence_plot(c, shap_values=shapvals, features=X, show=False, ax=ax1)
        shap.plots.scatter(shapvals[:, c], show=False, ax=ax1)
        f1.tight_layout()
        f1.savefig(expl_path + "dependence_{}.pdf".format(c))
    
    # Create beeswarm plot
    plt.clf()
    _ = shap.plots.beeswarm(shapvals, show=False, max_display=6)
    plt.tight_layout()
    plt.savefig(expl_path + "beeswarm.pdf")

plt.clf()

Exact explainer: 11741it [08:45, 22.03it/s]                                     
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 5837it [04:02, 23.21it/s]                                      
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 2918it [01:58, 22.60it/s]                                      
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 1165it [00:47, 19.42it/s]                                      
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 572it [00:24, 13.90it/s]                                       
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


### Ensemble approach

#### Cross-validation

In [14]:
# Placeholder for all preds just in case things go wrong
all_preds = []

# Iterate over all versions of the data
for n in tqdm(Ns):
    # Prepare ensemble data
    X, y = PrepareEnsembleData(mimic_imputed[n], yvar, covars=Xvars)
    
    # Construct base model
    # basemdl = RandomForestClassifier()
    
    # Run K-fold CV
    metrics, preds = KFoldEnsemble(n_splits, X, y, 
                                   mimic_imputed[n]["missingflag"].any(axis=1), 
                                   basemdl, classifier=True, random_state=SEED, 
                                   resample=True, pred_cutoff=pred_cutoff)
    
    # Store metrics
    all_preds.append(preds)
    mimic_auroc.loc[n, "Ensemble"] = metrics["AUROC"]
    mimic_f1["Ensemble"].loc[n] = metrics["F1"]

100%|█████████████████████████████████████████████| 5/5 [04:38<00:00, 55.62s/it]


#### Full model training and explanation

In [15]:
%matplotlib agg
# Iterate over all versions of the data
for n in Ns:
    # Prepare ensemble data
    X, y = PrepareEnsembleData(mimic_imputed[n], yvar, covars=Xvars)
    
    # Due to low % of readmissions, need to do resampling of minority class
    # only for model training
    index_pos = y[0][y[0] == 1].index
    index_neg = y[0][y[0] == 0].index
    index_pos_up = np.random.choice(
        index_pos, size=len(index_neg), replace=True
    )
    index_up = np.concatenate((index_pos_up, index_neg), axis=0)
    X_up, y_up = {}, {}
    for j in range(len(X)):
        X_up[j] = X[j].loc[index_up]
        y_up[j] = y[j].loc[index_up]
    
    # Construct ensemble model
    # basemdl = RandomForestClassifier()
    ensemblerf = EnsembleClassifier(basemdl).fit(X_up, y_up)
    
    # Create output path for model explanation
    expl_path = figures_path + "mimic_ensemble_{}/".format(n)
    if not os.path.exists(expl_path):
        os.mkdir(expl_path)
    
    # Get complete case data for model explanation
    mflag = mimic_imputed[n]["missingflag"].any(axis=1)
    Xobs = X[0][Xvars][~mflag]
    
    # Model explanation
    background_data = shap.maskers.Independent(Xobs, max_samples=100)
    pred_fn = lambda x: ensemblerf.predict_proba(x)[:, 1]
    expl = shap.Explainer(pred_fn, background_data, link=shap.links.logit)
    
    # Calculate SHAP values
    shapvals = expl(Xobs)
    
    # Create SHAP dependence plots for selected features
    for c in selected_expl:
        f1, ax1 = plt.subplots(figsize=(5, 5), ncols=1, nrows=1)
        # shap.dependence_plot(c, shap_values=shapvals, features=Xobs, show=False, ax=ax1)
        shap.plots.scatter(shapvals[:, c], show=False, ax=ax1)
        f1.tight_layout()
        f1.savefig(expl_path + "dependence_{}.pdf".format(c))
    
    # Create beeswarm plot
    plt.clf()
    _ = shap.plots.beeswarm(shapvals, show=False, max_display=6)
    plt.tight_layout()
    plt.savefig(expl_path + "beeswarm.pdf")

plt.clf()

Exact explainer: 11741it [1:55:45,  1.69it/s]                                   
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 5837it [55:07,  1.76it/s]                                      
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 2918it [26:21,  1.83it/s]                                      
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 1165it [10:55,  1.75it/s]                                      
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 572it [05:26,  1.69it/s]                                       
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


### Weighting approach

#### Cross-validation

In [16]:
# Placeholder for all preds just in case things go wrong
all_preds = []

# Iterate over all versions of the data
for n in tqdm(Ns):
    # Construct base model
    # basemdl = RandomForestClassifier()
    
    # Run K-fold CV
    metrics, preds = KFoldWeighted(n_splits, mimic_imputed[n], yvar, 
                                   basemdl, classifier=True, random_state=SEED, 
                                   covars=Xvars, pred_cutoff=pred_cutoff, resample=True)
    
    # Store metrics
    all_preds.append(preds)
    mimic_auroc.loc[n, "Weighting"] = metrics["AUROC"]
    mimic_f1["Weighting"].loc[n] = metrics["F1"]

100%|█████████████████████████████████████████████| 5/5 [02:06<00:00, 25.37s/it]


#### Full model training and explanation

In [17]:
%matplotlib agg
# Iterate over all versions of the data
for n in Ns:
    # Prepare weighted data
    X, y, w = PrepareWeightedData(mimic_imputed[n], yvar, covars=Xvars)
    
    # Due to low % of readmissions, need to do resampling of minority class
    # only for model training
    index_pos = y[y == 1].index
    index_neg = y[y == 0].index
    index_pos_up = np.random.choice(
        index_pos, size=len(index_neg), replace=True
    )
    index_up = np.concatenate((index_pos_up, index_neg), axis=0)
    X_up = X.loc[index_up]
    w_up = pd.Series(w, index=y.index).loc[index_up].values
    y_up = y.loc[index_up]
    
    # Construct weighted model
    weightedrf = copy.deepcopy(basemdl).fit(X_up, y_up, sample_weight=w_up)
    
    # Create output path for model explanation
    expl_path = figures_path + "mimic_weighting_{}/".format(n)
    if not os.path.exists(expl_path):
        os.mkdir(expl_path)
    
    # Get complete case data for model explanation
    mflag = mimic_imputed[n]["missingflag"].any(axis=1)
    Xobs = mimic_imputed[n]["imp"][0][Xvars][~mflag]
    
    # Model explanation
    background_data = shap.maskers.Independent(Xobs, max_samples=100)
    pred_fn = lambda x: weightedrf.predict_proba(x)[:, 1]
    expl = shap.Explainer(pred_fn, background_data, link=shap.links.logit)
    
    # Calculate SHAP values
    shapvals = expl(Xobs)
    
    # Create SHAP dependence plots for selected features
    for c in selected_expl:
        f1, ax1 = plt.subplots(figsize=(5, 5), ncols=1, nrows=1)
        # shap.dependence_plot(c, shap_values=shapvals, features=Xobs, show=False, ax=ax1)
        shap.plots.scatter(shapvals[:, c], show=False, ax=ax1)
        f1.tight_layout()
        f1.savefig(expl_path + "dependence_{}.pdf".format(c))
    
    # Create beeswarm plot
    plt.clf()
    _ = shap.plots.beeswarm(shapvals, show=False, max_display=6)
    plt.tight_layout()
    plt.savefig(expl_path + "beeswarm.pdf")

plt.clf()

Exact explainer: 11741it [07:11, 26.60it/s]                                     
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 5837it [03:22, 27.35it/s]                                      
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 2918it [01:36, 27.05it/s]                                      
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 1165it [00:39, 21.97it/s]                                      
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Exact explainer: 572it [00:19, 14.12it/s]                                       
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


### Summary

In [18]:
# Print all AUROCs
print(np.round(mimic_auroc, 3))

          CC  Ensemble  Weighting
20000  0.607     0.613      0.611
10000  0.559     0.586      0.583
5000   0.527     0.545      0.545
2000   0.518     0.540      0.557
1000   0.478     0.536      0.542


In [19]:
# Print all F1 scores (cutoff 0.1)
print(np.round(pd.concat(
    [mimic_f1["CC"][[0.1]].rename(columns={0.1: "CC"}), 
     mimic_f1["Ensemble"][[0.1]].rename(columns={0.1: "Ensemble"}), 
     mimic_f1["Weighting"][[0.1]].rename(columns={0.1: "Weighting"})],
    axis=1
), 3))

          CC  Ensemble  Weighting
20000  0.159     0.159      0.159
10000  0.156     0.157      0.156
5000   0.160     0.161      0.160
2000   0.156     0.159      0.157
1000   0.161     0.159      0.162


In [20]:
# Save all metrics in CSV
mimic_auroc.to_csv(results_path + "mimic_auroc_{}{}.csv".format(basemdlname, model_iter_name))
#mimic_f1.to_csv(results_path + "mimic_f1_{}{}.csv".format(basemdlname, model_iter_name))
for c in ["CC", "Ensemble", "Weighting"]:
    mimic_f1[c].to_csv(results_path + "mimic_f1_{}_{}{}.csv".format(
        c, basemdlname, model_iter_name
    ))